In [5]:
pip install tensorflow scikit-learn pandas

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

SEED = 42
tf.keras.utils.set_random_seed(SEED)
np.random.seed(SEED)

In [11]:
train_df = pd.read_json("train.jsonl", lines=True, encoding="utf-8")
test_df  = pd.read_csv("test.csv")
sample   = pd.read_csv("sample_submission.csv")

In [12]:
train_df["full_text"] = (train_df["headline"].astype(str) + " " + train_df["text"].astype(str)).str.strip()
test_df["full_text"]  = test_df["text"].astype(str)

X = train_df["full_text"]
y = train_df["label"]

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)


In [14]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1,4),   # até 4-gramas
    max_df=0.9,
    min_df=2,
    max_features=80000   # mais features para enriquecer
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf   = vectorizer.transform(X_val)
X_test_tfidf  = vectorizer.transform(test_df["full_text"])

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy'
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [16]:
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights_dict = {cls: w for cls, w in zip(classes, class_weights)}

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=3, restore_best_weights=True, monitor="val_loss"
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.5, patience=2, monitor="val_loss"
)

history = model.fit(
    X_train_tfidf, y_train,
    validation_data=(X_val_tfidf, y_val),
    epochs=20,
    batch_size=128,
    callbacks=[early_stop, reduce_lr],
    verbose=1,
    class_weight=class_weights_dict
)


Epoch 1/20
116/116 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - loss: 0.2968 - val_loss: 0.0381 - learning_rate: 0.0010
Epoch 2/20
 34/116 ━━━━━━━━━━━━━━━━━━━━ 1:21 988ms/step - loss: 0.0180

In [ ]:
val_probs = model.predict(X_val_tfidf).ravel()

best_thr, best_bal = 0.5, -1
for thr in np.linspace(0.3, 0.7, 41):  # testa de 0.30 até 0.70
    preds = (val_probs > thr).astype(int)
    bal = balanced_accuracy_score(y_val, preds)
    if bal > best_bal:
        best_bal, best_thr = bal, thr

print("Melhor threshold:", best_thr, "| Balanced Acc:", best_bal)
print(classification_report(y_val, (val_probs > best_thr).astype(int)))

In [ ]:
test_probs = model.predict(X_test_tfidf).ravel()
test_preds = (test_probs > best_thr).astype(int)

submission = sample.copy()
submission["label"] = test_preds
submission.to_csv("submission.csv", index=False)

print("Arquivo submission.csv gerado!")